In [1]:
import sys

sys.path.insert(0, '..')

from app.processing.stt.SIWhisperModel import SIWhisperModel
from app.SIWeaviateClient import SIWeaviateClient
from app.processing.YoutubeProcessor import YoutubeProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/google/api_core/exceptions.py:37: ImportWarning: Please install grpcio-status to obtain helpful grpc error messages.
  warnings.warn(
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
with SIWeaviateClient() as client:
    YoutubeProcessor(client=client, whisper=whisper, youtube_url="https://www.youtube.com/watch?v=QMJmmJqDL1w")

AUDIO CODEC aac


whisper-medium: 100%|██████████| 167.346/167.346 [00:10<00:00, 16.38s/s]


time_taken 10.6313955783844


In [ ]:
from collections import defaultdict
from weaviate.classes.query import Filter, GeoCoordinate, MetadataQuery, QueryReference, GroupBy
import weaviate
import weaviate.classes as wvc

client = weaviate.connect_to_local()
from weaviate.classes.query import QueryReference

def hybrid_search_chunks(query, limit=5):
    document_chunks = client.collections.get("DocumentChunk")
    
    response = document_chunks.query.hybrid(
        query=query,
        alpha=0.5,  # Adjust this value to balance between vector and keyword search
        limit=limit,
        return_metadata=["score"],
        return_references=QueryReference(
            link_on="belongsToDocument",
            return_properties=["document_id", "media_name"]
        ),
        group_by=GroupBy(
            prop="belongsToDocument",
            number_of_groups=limit,
            objects_per_group=3  # Adjust this as needed
        )
    )
    
    return response

hybrid_search_chunks("axolot")

In [ ]:
from weaviate.classes.query import (
    MetadataQuery,
    HybridFusion,
    GroupBy,
    Filter,
)
client = weaviate.connect_to_local()

def hybrid_search_and_group(search_term, limit=10):
    document_chunk = client.collections.get("DocumentChunk")
    
    response = document_chunk.query.hybrid(
        query=search_term,
        alpha=0.5,  # Adjust this value to balance between vector and keyword search
        fusion_type=HybridFusion.RELATIVE_SCORE,
        query_properties=[
            "text",
        ],
        limit=limit,
        return_metadata=MetadataQuery(
            distance=True,
            score=True,
        ),
        group_by=GroupBy(
            prop="belongsToDocument",
            number_of_groups=10,  # Adjust this value based on how many document groups you want
            objects_per_group=5,  # Adjust this value based on how many chunks per document you want
        ),
        # filters=Filter.by_property("belongsToDocument").not_equal(None),
    )

    return response

# Usage
search_results = hybrid_search_and_group("your search term here")
print("search_results", search_results)
# Process and print results
for group_name, group in search_results.groups.items():
    print(f"Document: {group_name}")
    print(f"Number of chunks: {group.number_of_objects}")
    for obj in group.objects:
        print(f"  Chunk: {obj.properties['text'][:50]}...")  # Print first 50 characters of each chunk
        print(f"  Score: {obj.metadata.score}")
        print(f"  Distance: {obj.metadata.distance}")
        print("---")
    print("=" * 50)

In [ ]:
client = weaviate.connect_to_local()


# document_chunk = client.collections.get("DocumentChunk")
# document = client.collections.get("Document")
# response = document_chunk.query.hybrid(
#     query="sniper",
#     # alpha=0.5,
#     query_properties=["text"],
#     return_references=[QueryReference(link_on="belongsToDocument", return_properties=["document_id", "media_name"])],
#     # limit=10,
#     group_by=GroupBy(
#         prop="belongsToDocument",
#         number_of_groups=10,
#         objects_per_group=5,
#     ),
# )

# print("\n\n==============\n\n")
# print(response)
# print("\n\n==============\n\n")
# for group_name, group in response.groups.items():
#     print(f"Document: {group_name}")
#     print(f"Number of chunks: {group.number_of_objects}")
#     for obj in group.objects:
#         print(f"  Chunk: {obj.properties['text'][:50]}...")  # Print first 50 characters of each chunk
#         print("---")
#     print("=" * 50)


group_by = GroupBy(
    prop="belongsToDocument.document_id",  # group by this cross-referenced property
    objects_per_group=3,  # maximum objects per group
    number_of_groups=2,  # maximum number of groups
)

# Perform the hybrid query
document_chunk = client.collections.get("DocumentChunk")
response = document_chunk.query.hybrid(
    alpha=0.75,
    query="your_search_query",  # replace with your actual search query
    group_by=group_by,
    return_references=[QueryReference(link_on="belongsToDocument.document_id")]  # include the cross-referenced property
)

client.close()
print(response)
# Print the grouped results
for grp_name, grp_content in response.groups.items():
    print(grp_name, grp_content.objects)

In [ ]:
client = weaviate.connect_to_local()

# Define the grouping parameters
group_by = GroupBy(
    prop="belongsToDocument.media_name",  # group by this cross-referenced property
    number_of_groups=10,  # maximum number of groups
    objects_per_group=5,  # maximum objects per group
)

# Perform the hybrid query
document_chunk = client.collections.get("DocumentChunk")
document = client.collections.get("Document")
response = document_chunk.query.hybrid(
    query="sniper",  # replace with your actual search query
    query_properties=["text"],  # specify the property to search
    return_references=[QueryReference(link_on="belongsToDocument", return_properties=["document_id", "media_name"])],  # include the cross-referenced properties
    group_by=group_by,
)

client.close()
# Print the grouped results
print(response)
for grp_name, grp_content in response.groups.items():
    print(grp_name, grp_content.objects)

In [13]:
import sys

sys.path.insert(0, '..')

import weaviate.classes as wvc
import weaviate
from collections import defaultdict
from app.schemas import ChunkWithScore, DocumentSearchResult
from weaviate.classes.query import Filter, GeoCoordinate, MetadataQuery, QueryReference, GroupBy


query = "l'île de Pâques"
client = weaviate.connect_to_local()
# print("SEARCH_MULTI_DOCUMENTS 0 ", query)
document_chunk = client.collections.get("DocumentChunk")
response = document_chunk.query.hybrid(
    query=query,
    return_metadata=wvc.query.MetadataQuery(score=True),
    query_properties=["text"],
    return_references=[QueryReference(link_on="belongsToDocument", return_properties=["document_id", "local_path", "original_public_path", "media_name", "max_score", "min_score"])]
)
# print("SEARCH_MULTI_DOCUMENTS 1 ", response)
client.close()

# Assume response is the result from the above query
grouped_results = defaultdict(list)
for chunk in response.objects:
    doc_ref = chunk.references['belongsToDocument'].objects[0].properties
    grouped_results[doc_ref['document_id']].append(chunk)

# Now `grouped_results` contains your data grouped by 'document_id'
# for document_id, chunks in grouped_results.items():
#     print(f"Media id: {document_id}, || Chunks: {chunks}")

documentsSearchResponse = []

for document_id, group in grouped_results.items():  # View by group
    # max_score = min(chunk for chunk in group)#group.max_score
    # min_score = min(chunk for chunk in group)#group.min_score
    currentDocumentChunksWithDistance = []
    for _documentChunk in group:
        score = _documentChunk.metadata.score
        cuurrentChunkWithScore = ChunkWithScore.model_validate({**_documentChunk.properties, "score": score})
        currentDocumentChunksWithDistance.append(cuurrentChunkWithScore)
    
    documentProperties = _documentChunk.references['belongsToDocument'].objects[0].properties
    
    documentSearchResult = DocumentSearchResult(
        document_id=documentProperties['document_id'],
        local_path=documentProperties['local_path'],
        original_public_path=documentProperties['original_public_path'],
        media_name=documentProperties['media_name'],
        min_score=min(chunk.score for chunk in currentDocumentChunksWithDistance),
        max_score=max(chunk.score for chunk in currentDocumentChunksWithDistance),
        chunks=currentDocumentChunksWithDistance
    )
    documentsSearchResponse.append(documentSearchResult)
for doc in documentsSearchResponse:
    print(doc.media_name)
    for chunk in doc.chunks:
        print(f"    -->  {chunk.score} | {chunk.text}")


Ile de Paques : la théorie de l’effondrement précolonial à nouveau démentie | Actu de science
    -->  1.0 | Une croyance encore bien ancrée dans les esprits laisse penser que jadis, les habitants de l'île de Pâques, de son nom autochtone Rapanoui, auraient surexploité leur environnement naturel, provoquant ainsi eux-mêmes leur disparition. Mais cette hypothèse a été maintes fois démentie et une nouvelle étude enfonce le clou.
    -->  0.7116864323616028 | En effet, l'île n'a jamais accueilli assez d'humains pour que cela ait entraîné leur Lorsque Rapa Nui est accosté par les colons européens, en 1722, il découvre d'imposantes statues de pierres érigées par milliers. Pourtant, il n'y a que 3 000 habitants sur place. Une question se pose alors, comment un si petit nombre d'individus a-t-il pu édifier de si nombreuses et si monumentales constructions ? Selon une idée popularisée par le géographe américain Jared Diamond il y a une vingtaine d'années, Rapa Nui était habité par de nombreux 

In [ ]:
response.objects[0].metadata

In [ ]:
from weaviate.classes.query import (
    MetadataQuery,
    HybridFusion,
    GroupBy,
    Filter,
)
from weaviate import Client
from weaviate.classes.config import Property, DataType, ReferenceProperty

# Initialize the client
client = Client("http://localhost:8080")
client.query \
   .get("DocumentChunk", ["text belongsToDocument {... on Document {document_id local_path} }"]) \
   .with_near_text({"concepts": ["biology"]}) \
    .with_group_by(["belongsToDocument"],groups=2,objects_per_group=2) \
   .with_limit(3) \
   .with_additional(['certainty']) \
   .do()
